**Mapas con folium e dataframes**

In [ ]:
# Carga datos de marcas desde un ficheiro e píntaas sobre un mapa

# Rede de bibliotecas de Galicia
# Descarga os datos das bibliotecas da rede de bibliotecas de Galicia do porta openData da Xunta
# https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0230/rede-bibliotecas-galicia

# Nota: Lembra que tes que engadir as marcas ao mapa unha a unha.


In [ ]:
import pandas as pd
import folium

In [ ]:
bibliotecas = pd.read_csv('https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0230/rede-bibliotecas-galicia/001/descarga-directa-ficheiro.csv',sep=';')
bibliotecas.head()

In [ ]:
# Os datos máis importantes que vamos utilizar son os das coordenadas, polo que hai que
# asegurarse de que os datos están limpos e van funcionar

# Algúns problemas que nos podemos encontrar son:
# - valores nulos
# - espazos en branco
# - valores mal codificados
# - valores erróneos
# - columnas desprazadas

# En función do problema e dos nosos obxecitivos poderemos:
# - corrixir os datos
# - limpar os datos
# - eliminar as filas
# - etc...

In [ ]:
# Encontramos valores nulos
bibliotecas[bibliotecas.COORDENADAS.isnull()]
# Poderiamos eliminar a liña, xa que é unha sóa, ou buscar os datos
# bibliotecas.drop(index= bibliotecas[bibliotecas.COORDENADAS.isnull()].index, inplace=True)
# bibliotecas.reset_index(drop=True,inplace=True)

In [ ]:
# Neste caso encontrámonos con diferentes problemas que dificultarían o noso traballo:
# - hai espazos en branco entre as coordenadas
# - hai valores erróneos
# - etccc

In [ ]:
bibliotecas[bibliotecas['COORDENADAS'].str.match(r'^[0-9]+\.[0-9]+\,[+\-]*[0-9]+\.[0-9]+$')==False]

In [ ]:
# Unha moi boa solución para o caso das coordenadas, e tamén para moitos outros, é utilizar EXPRESIÓNS REGULARES
# Utilizando expresións regulares podemos indicar como queremos que sexan os datos exactamente

# Creamos un novo dataframe SÓ COAS COORDENADAS CORRECTAS
biblios = bibliotecas[bibliotecas['COORDENADAS'].str.match(r'^[0-9]+\.[0-9]+\,[+\-]*[0-9]+\.[0-9]+$')==True].copy()

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=8, width=600, height=600)

for index, biblioteca in biblios.iterrows():
    folium.Marker([biblioteca['COORDENADAS'].split(',')[0], biblioteca['COORDENADAS'].split(',')[1]]).add_to(m)
    #folium.Marker([biblioteca['COORDENADAS'].split(',')[0], biblioteca['COORDENADAS'].split(',')[1]],popup=biblioteca['NOME']).add_to(m)
    #folium.Marker([biblioteca['COORDENADAS'].split(',')[0], biblioteca['COORDENADAS'].split(',')[1]],popup=biblioteca['NOME'],icon=folium.Icon(icon='book')).add_to(m)
m

In [ ]:
len(bibliotecas)

**Clústers de marcas**

Cando temos moitas marcas nun territorio pode ser interesante utilizar clústers.

In [ ]:
# A diferenza que nun mapa "standar", nesta ocasión creamos un clúster e engadímolo ao mapa
# A continuación engadimos as marcas ao clúster, en lugar de engadilo ao mapa

m = folium.Map(location=[43, -8.20],zoom_start=8, width=600, height=600)

marker_cluster = folium.plugins.MarkerCluster().add_to(m)

for index, biblioteca in biblios.iterrows():
    folium.Marker([biblioteca['COORDENADAS'].split(',')[0], biblioteca['COORDENADAS'].split(',')[1]],popup=biblioteca['NOME'],icon=folium.Icon(icon='book')).add_to(marker_cluster)

m

**Ficheiros KMZ/KML**

In [ ]:
# No portal ABERTOS da Xunta tamén encontramos os datos en formato KML
# https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0230/rede-bibliotecas-galicia

# Ficheiros KMZ e KML
# Trátase de formatos popularizados por Google (Earth/Maps) para representar información xeográfica
# KML é un ficheiro XML
# KMZ é un ficheiroz ZIP onde nos encontramos un doc.kml e outros ficheiros con información extra

# Os campos interesantes para nós nos ficheiros KML son:
# <Placemark>
#   <name>
#   <Location>
#       <longitude>
#       <latitude>

In [ ]:
# Ao tratarse de ficheiros XML podemos parsealos con Beautifulsoup
# Ficheiro KML
url = 'https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0230/rede-bibliotecas-galicia/002/descarga-directa-ficheiro.kml'

import requests
import lxml # Pode ser necesario instalar o parser xml
from bs4 import BeautifulSoup

response = requests.get(url)
#print(response.text)
soup = BeautifulSoup(response.content,'lxml')

In [ ]:
# A información de cada biblioteca está nun único "placemark"
# Creamos unha lista de todas as bibliotecas
bibliotecas = soup.find_all('placemark')
len(bibliotecas)

In [ ]:
# Visualizamos unha biblioteca
bibliotecas[0]

In [ ]:
# vemos que as coordenadas están "ao revés" do que espera folium (lonxitude,latitude)
bibliotecas[0].find('coordinates').text

In [ ]:
for biblioteca in bibliotecas:
    print(biblioteca.find('name').text)
    print(biblioteca.find('coordinates').text)

In [ ]:
# No KML tamén pode haber erros nas coordenadas
# Poderíamos construír un dataframe cos datos e pintar o mapa do mesmo xeito que antes
# Tamén podemos iterar os datos e ir pintando o mapa
# En lugar de utilizar a función "match" de pandas.series usaremos a función search do módulo 're'
import re

m = folium.Map(location=[43, -8.20],zoom_start=8, width=600, height=600)

#^[0-9]+\.[0-9]+\,[+\-]*[0-9]+\.[0-9]+$

for biblioteca in bibliotecas:
    if (re.search('^[+\-]*[0-9]+\.[0-9]+\,[0-9]+\.[0-9]+$',biblioteca.find('coordinates').text)):
        folium.Marker([biblioteca.find('coordinates').text.split(',')[1], biblioteca.find('coordinates').text.split(',')[0]]).add_to(m)        
m